### Convert the P1-03-TTD_crossmatching_filtered.txt file to P1-03-TTD_crossmatching_filtered.csv file

In [1]:
import csv

def process_input_file(input_file, output_file):
    # Open input file for reading
    with open(input_file, 'r') as f:
        lines = f.readlines()

    # Initialize variables
    current_drug = {}
    data = []

    # List of all possible fields
    fieldnames = ['TTDDRUID', 'DRUGNAME', 'D_FOMULA', 'PUBCHCID', 'PUBCHSID', 'CASNUMBE', 'CHEBI_ID', 'SUPDRATC', 'SUPDRCAS']

    # Process each line in the input file
    for line in lines:
        parts = line.split('\t')
        if len(parts) < 2:
            continue  # skip lines that don't have enough parts
        key = parts[0].strip()
        value = parts[1].strip()
        if key in fieldnames:
            current_drug[key] = value
        if key == 'SUPDRCAS':
            # Assuming that each drug entry ends after COMPCLAS
            # Initialize missing fields with empty strings
            for field in fieldnames:
                if field not in current_drug:
                    current_drug[field] = ''
            data.append(current_drug.copy())
            current_drug = {}

    # Write data to CSV file
    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        # Write header
        writer.writeheader()
        
        # Write rows
        for drug in data:
            writer.writerow(drug)

# Example usage
input_file = '/data4/msc23104470/TTD/raw_data_to_csv/P1-03-TTD_crossmatching_filtered.txt'
output_file = '/data4/msc23104470/TTD/P1-03-TTD_crossmatching_filtered.csv'
process_input_file(input_file, output_file)

In [2]:
import pandas as pd
df_path = '/data4/msc23104470/TTD/P1-03-TTD_crossmatching_filtered.csv'
df = pd.read_csv(df_path, sep=',')

In [10]:
df

,TTDDRUID,DRUGNAME,D_FOMULA,PUBCHCID,PUBCHSID,CASNUMBE,CHEBI_ID,SUPDRATC,SUPDRCAS
0,D00AMQ,Ethanol,C2H6O,702,2694; 3752; 122577; 588484; 601786; 825155; 82...,CAS 64-17-5,CHEBI:16236,D08AX08; V03AB16; V03AZ01,cas=000051412
1,D00AOJ,Docosanol,C22H46O,12620,74102; 588002; 616715; 3137145; 4438509; 78503...,CAS 661-19-8,CHEBI:31000,D06BB11,cas=000661198
2,D00AWT,Propericiazine,C21H23N3OS,4747,7703132; 7848548; 7980281; 8150082; 8152913; 1...,CAS 2622-26-6,CHEBI:31981,N05AC01,cas=002622266
3,D00AXJ,Miconazole,C18H14Cl4N2O,4189,443253; 602748; 853814; 5025447; 7847482; 7979...,CAS 22916-47-8,CHEBI:82892,A01AB09; A07AC01; D01AC02; G01AF04; J02AB01; S...,cas=022916478
4,D00BCG,Goserelin,C59H84N18O14,5311128,7979432; 12012688; 14841899; 39340846; 4650733...,CAS 65807-02-5,CHEBI:5523,L02AE03,cas=065807025
...,...,...,...,...,...,...,...,...,...
1206,D0X4NX,Polyporic acid,C18H12O4,11056,98392; 605348; 5082610; 8138243; 8158049; 1546...,CAS 548-59-4,CHEBI:144197,A02BC05,cas=161973100
1207,D0X8KC,Arsenite,AsHO2,24577,623501; 5341080; 8167965; 24705099; 29291640; ...,CAS 13768-07-5,CHEBI:93427,L01XX27,cas=001327533
1208,D0XU1I,G3139 + Dacarbazine,C15H20N4O3S,91865905; 5353562,3153136; 7979427; 11056888; 14884365; 26755743...,CAS 139298-40-1,CHEBI:91460,L01AX04,cas=004342034
1209,D0XU9F,G3139 + cytarabine (ARA-C),C22H30F3N5O8,91865905; 6253,NaN,CAS 383907-43-5,CHEBI:92053,L01BC01,cas=000147944


### Drop extra columns
Dropped the extra columns and only kept PUBCHCID, which is saved in a separated 'CSV' file named P1-03-TTD_CIDs.csv. Then used a python script (below) to download the InChI key for PubChem CIDs from the PubChem server.
The script was run on Lugh as a batch file. The code is below for documentation purpose.

### Read the downloaded file into a dataframe and merge with P1-03-TTD_crossmatching_filtered.csv

In [1]:
import pandas as pd
import numpy as np
crossmatch_file_path = '/data4/msc23104470/TTD/P1-03-TTD_crossmatching_filtered.csv'
CID_to_InChIKey_path = '/data4/msc23104470/TTD/P1-03-TTD_CID_to_InChIKey.csv'

crossmatch_file = pd.read_csv(crossmatch_file_path, sep=',')
CID_to_InChIKey = pd.read_csv(CID_to_InChIKey_path, sep = ',')

In [2]:
CID_to_InChIKey.dtypes

CID          int64
InChIKey    object
dtype: object

In [3]:
crossmatch_file['CID'] = pd.to_numeric(crossmatch_file['CID'], errors='coerce').astype('Int64')

In [4]:
merged_df = pd.merge(crossmatch_file, CID_to_InChIKey, on='CID', how='left')

MergeError: Passing 'suffixes' which cause duplicate columns {'InChIKey_x'} is not allowed.

In [12]:
merged_df.to_csv('/data4/msc23104470/TTD/P1-03-TTD_crossmatching_filtered.csv', sep=',')

Now, `P1-05-Drug_disease_ALL.csv` and `merged_df` have TTDDRUID in common. We can merge these two dataframes based on this column to figure out how many drugs we have that we have all the information for.

In [15]:
ALL_path = '/data4/msc23104470/TTD/P1-05-Drug_disease_ALL.csv'

ALL = pd.read_csv(ALL_path, sep=',')

In [16]:
merged_df = pd.merge(merged_df, ALL, on='TTDDRUID', how='left')

In [14]:
merged_df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)

In [12]:
merged_df_path = '/data4/msc23104470/TTD/P1-03-TTD_crossmatching_filtered.csv'
merged_df = pd.read_csv(merged_df_path, sep=',')
merged_df.drop(columns=['Unnamed: 0.2','Unnamed: 0.1','Unnamed: 0','InChIKey_x','InChIKey_y'], inplace=True)

In [17]:
merged_df

,TTDDRUID,DRUGNAME_x,D_FOMULA,CID,PUBCHSID,CASNUMBE,CHEBI_ID,SUPDRATC,SUPDRCAS,InChIKey,DRUGNAME_y,INDICATI,ICD-11,CLINICAL_STATUS
0,D00AMQ,Ethanol,C2H6O,702.0,2694; 3752; 122577; 588484; 601786; 825155; 82...,CAS 64-17-5,CHEBI:16236,D08AX08; V03AB16; V03AZ01,cas=000051412,LFQSCWFLJHTTHZ-UHFFFAOYSA-N,Ethanol,Cystitis,GC00,Approved
1,D00AMQ,Ethanol,C2H6O,702.0,2694; 3752; 122577; 588484; 601786; 825155; 82...,CAS 64-17-5,CHEBI:16236,D08AX08; V03AB16; V03AZ01,cas=000051412,LFQSCWFLJHTTHZ-UHFFFAOYSA-N,Ethanol,Chronic pain,MG30,Approved
2,D00AOJ,Docosanol,C22H46O,12620.0,74102; 588002; 616715; 3137145; 4438509; 78503...,CAS 661-19-8,CHEBI:31000,D06BB11,cas=000661198,NOPFSRXAKWQILS-UHFFFAOYSA-N,Docosanol,Herpes simplex virus infection,1F00,Approved
3,D00AWT,Propericiazine,C21H23N3OS,4747.0,7703132; 7848548; 7980281; 8150082; 8152913; 1...,CAS 2622-26-6,CHEBI:31981,N05AC01,cas=002622266,LUALIOATIOESLM-UHFFFAOYSA-N,Propericiazine,Psychiatric disorder,6E8Z,Approved
4,D00AXJ,Miconazole,C18H14Cl4N2O,4189.0,443253; 602748; 853814; 5025447; 7847482; 7979...,CAS 22916-47-8,CHEBI:82892,A01AB09; A07AC01; D01AC02; G01AF04; J02AB01; S...,cas=022916478,BYBLEWFAAKGYCD-UHFFFAOYSA-N,Miconazole,Fungal infection,1F29-1F2F,Approved
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,D0X4NX,Polyporic acid,C18H12O4,11056.0,98392; 605348; 5082610; 8138243; 8158049; 1546...,CAS 548-59-4,CHEBI:144197,A02BC05,cas=161973100,HZKFHDXTSAYOSN-UHFFFAOYSA-N,NaN,NaN,NaN,NaN
1507,D0X8KC,Arsenite,AsHO2,24577.0,623501; 5341080; 8167965; 24705099; 29291640; ...,CAS 13768-07-5,CHEBI:93427,L01XX27,cas=001327533,XHTRYVMKFMUJLU-UHFFFAOYSA-N,NaN,NaN,NaN,NaN
1508,D0XU1I,G3139 + Dacarbazine,C15H20N4O3S,NaN,3153136; 7979427; 11056888; 14884365; 26755743...,CAS 139298-40-1,CHEBI:91460,L01AX04,cas=004342034,NaN,G3139 + Dacarbazine,Melanoma,2C30,Investigative
1509,D0XU9F,G3139 + cytarabine (ARA-C),C22H30F3N5O8,NaN,NaN,CAS 383907-43-5,CHEBI:92053,L01BC01,cas=000147944,NaN,G3139 + cytarabine (ARA-C),Acute leukaemia,2A60,Investigative


In [19]:
### write it to a file
merged_df.to_csv('/data4/msc23104470/TTD/P1-03-TTD_crossmatching_filtered.csv', sep=',')

Now, merge the `P1-03-TTD_crossmatching_filtered.csv` with LINCS dataset..

In [2]:
import pandas as pd
data_matrix_path = '/data4/msc23104470/final_project/preprocessing/L1000-FWD/output/data-matrix-w-HUGO-symbols.csv'
crossmatching_file_path = '/data4/msc23104470/TTD/P1-03-TTD_crossmatching_filtered.csv'

data_matrix = pd.read_csv(data_matrix_path, sep=',')
crossmatching = pd.read_csv(crossmatching_file_path, sep=',')

In [5]:
import pandas as pd
import re

# Define a function to extract the pattern
def extract_brd(text):
    match = re.search(r'BRD-[A-Za-z0-9]{9}', text)
    if match:
        return match.group(0)
    else:
        return None

# Apply the function to the 'rid' column
data_matrix['BRD ID'] = data_matrix['rid'].apply(extract_brd)

Now, map the InChI key with the last column of `data_matrix` i.e., BRD ID with all_brd_inchi.txt

In [7]:
all_brd_path = '/data4/msc23104470/final_project/preprocessing/adb1019/data-for-remaining-33-brdids_none-found/all_brd_inchi.txt'
all_brd = pd.read_csv(all_brd_path, sep='\t')

In [10]:
mapped_df = pd.merge(data_matrix, all_brd, on='BRD ID', how='left')

Now, all of the InChI Keys are mapped with the LINCS dataset with `0` null value in the `InChI Key` column..

Now, map the TTD dataset here on=`InChI Key`

In [20]:
TTD_dataset_path = '/data4/msc23104470/TTD/P1-03-TTD_crossmatching_filtered.csv'
TTD_dataset = pd.read_csv(TTD_dataset_path, sep=',')

In [24]:
mapped_TTD_LINCS_df = pd.merge(mapped_df, TTD_dataset, on='InChI Key', how='left')

In [26]:
mapped_df

,Unnamed: 0,rid,PSME1,ATF1,RHEB,FOXO3,RHOA,IL1B,ASAH1,RALA,...,SQRDL,HEBP1,ATP11B,CD320,MLLT11,CEBPZ,CBR3,ATC Codes,BRD ID,InChI Key
0,0,AML001_CD34_6H:BRD-K43389675:10,0.004217,-0.018644,0.019958,-0.056607,-0.017211,-0.040410,-0.012175,-0.013127,...,0.035868,0.012153,0.036431,0.001324,0.039140,-0.055924,-0.012614,L01DB02; L01XY01,BRD-K43389675,STQGQHZAVUOBTE-VGBVRHCVSA-N
1,1,AML001_PC3_6H:BRD-A45664787:10,-0.000762,-0.000233,0.003371,0.004516,0.009972,0.130947,-0.024684,0.017885,...,-0.007869,0.029858,0.012888,-0.052345,-0.008905,0.045641,-0.023661,B01AC11,BRD-A45664787,HIFJCPQKFCZDDL-ACWOEMLNSA-N
2,2,AML001_PC3_6H:BRD-K27316855:0.37037,-0.002200,0.006644,-0.010455,0.034244,-0.000161,-0.015078,-0.024642,-0.033437,...,0.031202,0.002686,0.006412,-0.021728,0.020117,0.019046,0.026748,D05AX03; A11CC04,BRD-K27316855,GMRQFYUYWCNGIN-NKMMMXOESA-N
3,3,AML001_PC3_6H:BRD-K27316855:1.11111,-0.001177,-0.003849,-0.009299,0.016905,0.007201,-0.021086,-0.023131,0.013359,...,0.011303,0.021753,0.025465,-0.006530,0.016000,0.017854,0.001950,D05AX03; A11CC04,BRD-K27316855,GMRQFYUYWCNGIN-NKMMMXOESA-N
4,4,AML001_PC3_6H:BRD-K27316855:10,0.007566,-0.000217,0.001623,-0.009524,-0.012488,-0.082372,-0.034615,0.002367,...,-0.004452,0.003234,-0.034610,0.032883,-0.001317,-0.036477,0.019521,D05AX03; A11CC04,BRD-K27316855,GMRQFYUYWCNGIN-NKMMMXOESA-N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12642,12587,RAD001_PC3_6H:BRD-K81418486-001-18-6:10,0.002199,-0.066665,0.005500,0.023552,0.000938,-0.015076,-0.010938,0.040864,...,0.036423,0.069621,-0.000006,0.011282,-0.042105,-0.018672,-0.127148,L01XH01,BRD-K81418486,WAEXFXRVDQXREF-UHFFFAOYSA-N
12643,12588,RAD001_PC3_6H:BRD-K81418486-001-18-6:3.3333,0.021221,-0.053229,-0.005322,0.010113,-0.000651,-0.009379,0.004355,0.015548,...,-0.033875,-0.020906,-0.041242,0.009414,-0.028997,0.068181,-0.031987,L01XH01,BRD-K81418486,WAEXFXRVDQXREF-UHFFFAOYSA-N
12644,12589,RAD001_PC3_6H:BRD-K81418486:10,-0.009087,-0.012380,0.023854,0.051941,0.031735,0.070593,-0.001130,0.013164,...,-0.002661,-0.031886,0.029401,0.035857,-0.017159,-0.064174,-0.010241,L01XH01,BRD-K81418486,WAEXFXRVDQXREF-UHFFFAOYSA-N
12645,12590,RAD001_PC3_6H:BRD-K84937637-001-03-2:0.1235,-0.001343,0.022330,0.011539,0.005044,0.017594,0.009275,-0.007178,-0.028721,...,0.009009,0.012971,0.001469,0.013327,-0.008807,-0.037837,0.022568,L01EG04; S01XA23; L04AH01,BRD-K84937637,QFJCIRLUMZQUOT-HPLJOQBZSA-N


In [27]:
TTD_dataset

,Unnamed: 0,TTDDRUID,DRUGNAME_x,D_FOMULA,CID,PUBCHSID,CASNUMBE,CHEBI_ID,SUPDRATC,SUPDRCAS,InChI Key,DRUGNAME_y,INDICATI,ICD-11,CLINICAL_STATUS
0,0,D00AMQ,Ethanol,C2H6O,702.0,2694; 3752; 122577; 588484; 601786; 825155; 82...,CAS 64-17-5,CHEBI:16236,D08AX08; V03AB16; V03AZ01,cas=000051412,LFQSCWFLJHTTHZ-UHFFFAOYSA-N,Ethanol,Cystitis,GC00,Approved
1,1,D00AMQ,Ethanol,C2H6O,702.0,2694; 3752; 122577; 588484; 601786; 825155; 82...,CAS 64-17-5,CHEBI:16236,D08AX08; V03AB16; V03AZ01,cas=000051412,LFQSCWFLJHTTHZ-UHFFFAOYSA-N,Ethanol,Chronic pain,MG30,Approved
2,2,D00AOJ,Docosanol,C22H46O,12620.0,74102; 588002; 616715; 3137145; 4438509; 78503...,CAS 661-19-8,CHEBI:31000,D06BB11,cas=000661198,NOPFSRXAKWQILS-UHFFFAOYSA-N,Docosanol,Herpes simplex virus infection,1F00,Approved
3,3,D00AWT,Propericiazine,C21H23N3OS,4747.0,7703132; 7848548; 7980281; 8150082; 8152913; 1...,CAS 2622-26-6,CHEBI:31981,N05AC01,cas=002622266,LUALIOATIOESLM-UHFFFAOYSA-N,Propericiazine,Psychiatric disorder,6E8Z,Approved
4,4,D00AXJ,Miconazole,C18H14Cl4N2O,4189.0,443253; 602748; 853814; 5025447; 7847482; 7979...,CAS 22916-47-8,CHEBI:82892,A01AB09; A07AC01; D01AC02; G01AF04; J02AB01; S...,cas=022916478,BYBLEWFAAKGYCD-UHFFFAOYSA-N,Miconazole,Fungal infection,1F29-1F2F,Approved
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,1506,D0X4NX,Polyporic acid,C18H12O4,11056.0,98392; 605348; 5082610; 8138243; 8158049; 1546...,CAS 548-59-4,CHEBI:144197,A02BC05,cas=161973100,HZKFHDXTSAYOSN-UHFFFAOYSA-N,NaN,NaN,NaN,NaN
1507,1507,D0X8KC,Arsenite,AsHO2,24577.0,623501; 5341080; 8167965; 24705099; 29291640; ...,CAS 13768-07-5,CHEBI:93427,L01XX27,cas=001327533,XHTRYVMKFMUJLU-UHFFFAOYSA-N,NaN,NaN,NaN,NaN
1508,1508,D0XU1I,G3139 + Dacarbazine,C15H20N4O3S,NaN,3153136; 7979427; 11056888; 14884365; 26755743...,CAS 139298-40-1,CHEBI:91460,L01AX04,cas=004342034,NaN,G3139 + Dacarbazine,Melanoma,2C30,Investigative
1509,1509,D0XU9F,G3139 + cytarabine (ARA-C),C22H30F3N5O8,NaN,NaN,CAS 383907-43-5,CHEBI:92053,L01BC01,cas=000147944,NaN,G3139 + cytarabine (ARA-C),Acute leukaemia,2A60,Investigative


In [32]:
null_count = mapped_TTD_LINCS_df['InChI Key'].isnull().sum()
null_count

0

Save the final dataframe to a `CSV` file..

In [33]:
mapped_TTD_LINCS_df.to_csv('/data4/msc23104470/TTD/mapped_TTD_LINCS_df.csv', sep=',')

### Performing exploratory data analysis

In [37]:
mapped_TTD_LINCS_df.drop(columns=['Unnamed: 0_x'], inplace=True)

In [39]:
mapped_TTD_LINCS_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14285 entries, 0 to 14284
Columns: 996 entries, rid to CLINICAL_STATUS
dtypes: float64(980), object(16)
memory usage: 108.6+ MB


In [40]:
mapped_TTD_LINCS_df.describe()

,PSME1,ATF1,RHEB,FOXO3,RHOA,IL1B,ASAH1,RALA,ARHGEF12,SOX2,...,UGDH,SQRDL,HEBP1,ATP11B,CD320,MLLT11,CEBPZ,CBR3,Unnamed: 0_y,CID
count,14285.000000,14285.000000,14285.000000,14285.000000,14285.000000,14285.000000,14285.000000,14285.000000,14285.000000,14285.000000,...,14285.000000,14285.000000,14285.000000,14285.000000,14285.000000,14285.000000,14285.000000,14285.000000,5775.000000,5.775000e+03
mean,0.001813,-0.002992,0.000051,0.003164,0.001144,0.000644,0.002504,0.001387,0.000693,-0.000494,...,-0.000976,0.000171,0.002073,0.000969,0.000450,-0.003302,-0.001232,-0.004609,637.664589,2.891454e+06
std,0.016535,0.023796,0.012913,0.029695,0.019992,0.031633,0.024331,0.019447,0.022661,0.019346,...,0.045247,0.037858,0.038228,0.033521,0.034054,0.041448,0.026699,0.043865,420.486869,1.694544e+07
min,-0.117327,-0.122305,-0.204829,-0.215167,-0.130489,-0.263849,-0.159729,-0.114276,-0.115586,-0.133574,...,-0.353697,-0.297680,-0.191928,-0.293076,-0.327162,-0.284894,-0.196601,-0.278027,7.000000,2.370000e+02
25%,-0.008433,-0.015841,-0.006944,-0.014006,-0.010894,-0.013675,-0.012183,-0.010692,-0.012747,-0.011786,...,-0.023966,-0.021763,-0.021272,-0.017578,-0.018081,-0.026666,-0.017576,-0.028842,225.000000,4.055000e+03
50%,0.001498,-0.001434,0.000094,0.002318,0.000296,-0.000676,0.001897,0.001723,0.000253,-0.001143,...,0.002649,-0.002334,0.001097,0.000934,0.001563,-0.002932,-0.001497,-0.002696,593.000000,5.591000e+03
75%,0.011829,0.012156,0.007279,0.020122,0.012354,0.013520,0.017224,0.013693,0.013784,0.009987,...,0.025760,0.017568,0.024103,0.019793,0.021002,0.019446,0.014927,0.021838,1041.000000,1.236310e+05
max,0.105239,0.089443,0.072280,0.177819,0.103859,0.336550,0.135837,0.102601,0.155566,0.195646,...,0.201445,0.440589,0.210098,0.198507,0.184664,0.268372,0.149663,0.275133,1502.000000,1.354095e+08


In [47]:
# number of unique values in the dataframe
mapped_TTD_LINCS_df.nunique()

rid                12574
PSME1              12572
ATF1               12570
RHEB               12569
FOXO3              12574
                   ...  
SUPDRCAS             471
DRUGNAME_y           467
INDICATI             255
ICD-11               230
CLINICAL_STATUS       15
Length: 996, dtype: int64

In [46]:
# number of missing values in the dataframe
mapped_TTD_LINCS_df.isnull().sum()

rid                   0
PSME1                 0
ATF1                  0
RHEB                  0
FOXO3                 0
                   ... 
SUPDRCAS           8510
DRUGNAME_y         8585
INDICATI           8585
ICD-11             8585
CLINICAL_STATUS    8585
Length: 996, dtype: int64

In [45]:
# percentage of missing value in the dataframe
(mapped_TTD_LINCS_df.isnull().sum()/(len(mapped_TTD_LINCS_df)))*100

rid                 0.000000
PSME1               0.000000
ATF1                0.000000
RHEB                0.000000
FOXO3               0.000000
                     ...    
SUPDRCAS           59.572979
DRUGNAME_y         60.098005
INDICATI           60.098005
ICD-11             60.098005
CLINICAL_STATUS    60.098005
Length: 996, dtype: float64

In [49]:
unique_count = mapped_TTD_LINCS_df['TTDDRUID'].nunique()
unique_count

474